# Mango Attribute Predict Model

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from IPython import get_ipython
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
import os

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.optim as optim
import missingno as msno

### Data Preprocessing

In [ ]:
df = pd.read_csv("../datathon-fme-mango/archive/product_data.csv")
attribute_data = pd.read_csv("../datathon-fme-mango/archive/attribute_data.csv")
print(df.head(), attribute_data.head())

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Pivot the attribute_data dataframe
attribute_pivot = attribute_data.pivot(index='cod_modelo_color', columns='attribute_name', values='des_value')

# Reset the index to merge with df
attribute_pivot.reset_index(inplace=True)

# Merge the pivoted attribute data with the product data
df = pd.merge(df, attribute_pivot, on='cod_modelo_color', how='left')

# Display the merged dataframe
print(df.head())

In [ ]:
# Check for missing values in the dataframe
missing_values = df.isnull().sum()

# Create a heatmap of the missing values
plt.figure(figsize=(12, 8))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Heatmap of Missing Values in df')
plt.show()